In [3]:
from keplergl import KeplerGl
import geopandas as gp
import pandas as pd
import numpy as np
from multiprocessing import Pool
from elasticsearch import Elasticsearch, helpers

STATE_LENGTH, COUNTY_LENGTH, CT_LENGTH, CBG_LENGTH, CB_LENGTH = 2, 5, 11, 12, 15
# Port 9243 vs 443: https://stackoverflow.com/questions/53829218/why-does-elastic-cloud-provides-a-specialized-port-9243-instead-of-only-using
ES_DEV = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:443'], 
                    http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'), timeout=30, max_retries=10, retry_on_timeout=True)
### Should start with with SF57 (before, SF52 only) # 50 states + DC + 6 territories (PR, VI, AS, GU, MP, UM): the more data coverage the better
SF57 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
SF57R = {value : key for (key, value) in SF57.items()}
# 50 states + DC + PR + VI
SF53 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR', '78': 'VI'}
SF53R = {value : key for (key, value) in SF53.items()}
# 50 states + DC + PR
SF52 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}
SF52R = {value : key for (key, value) in SF52.items()}
# 50 states + DC
SF51 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'}
SF51R = {value : key for (key, value) in SF51.items()}

# 8764 OZs total, as of Notice 2019-42, July 2019!
- https://www.cdfifund.gov/sites/cdfi/files/documents/designated-qozs.12.14.18.xlsx : provide OZ Tract Type, either:
    - Low-Income Community
    - Non-LIC Contiguous
- TIGER: provides shapefiles

# Overlay to confirm OZ tracts are subset of 2019 census tracts (same shapes)

In [ ]:
df_geom = gp.read_file("https://www.cdfifund.gov/sites/cdfi/files/documents/opportunity-zones=8764.-9-10-2019.zip")
df_geom = df_geom.sort_values(['STATENAME', 'COUNTYNAME'])
df_geom.shape, type(df_geom)

((8764, 4), geopandas.geodataframe.GeoDataFrame)

In [315]:
# OZ: alabama tracts geom
# al_oz = df_geom.query("STATENAME == 'Alabama'").copy()
oz_mp = df_geom.query("STATENAME == 'Northern Mariana Islands'").copy()
type(oz_mp), oz_mp.shape

(geopandas.geodataframe.GeoDataFrame, (20, 4))

In [317]:
# TIGER: all census tracts geom
# al_tracts = gp.read_file('https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_01_tract.zip')
tiger_mp = gp.read_file('https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_69_tract.zip')
type(tiger_mp), tiger_mp.shape

(geopandas.geodataframe.GeoDataFrame, (25, 13))

In [318]:
map1 = KeplerGl(height = 500)
map1.add_data(data=oz_mp[['CENSUSTRAC', 'geometry']], name='oz')
map1.add_data(data=tiger_mp[['GEOID', 'geometry']], name='all')
map1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'oz': {'index': [2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, …

### CONCLUSION: same tracts! 

## UPDATE bossdata mapping

In [477]:
### DOWNLOAD ALL GEOID from BOSSDATA SF56
def download_ES(sf):
    index_name = f"bossdata{sf}"
    match_query = {
        "query": {"match_all": {}},
        "_source": {
            "includes": "GEOID"
        },
    }
    res_gen = helpers.scan(ES_DEV, query= match_query, index= index_name) 
    print('Completed downloading ES index ', index_name)
    data = [record['_source']['GEOID'] for record in res_gen]
    np.savetxt(f'cb-ids/{sf}.txt', data, fmt="%s")
    print(f'Saved all geoids {index_name}, {len(data)=} to file')

    
def update_ES_mapping(sf):
    index_name = f"bossdata{sf}"
    update_request_body = {
        "properties": {
            ## Without reindex => RequestError(400, 'illegal_argument_exception', 'mapper [tractTypeOZ] cannot be changed from type [text] to [keyword]')
            ## "tractTypeOZ" :           {  "type": "keyword",  "null_value": "isNotOZ" },
            
            ## null_value i.e. set 0 as default value for the field -> allow indexing and searching https://www.elastic.co/guide/en/elasticsearch/reference/current/null-value.html
            "tractTypeKeywordOZ" :           {  "type": "keyword",  "null_value": "isNotOZ" },
            
        }
    }
    ES_DEV.indices.put_mapping(index = index_name, body = update_request_body)
    print('updated index ', index_name)
    
## OPTIONAL: DELETE a field from all documents; then if no reindex is done, best to forget about that field
## Doesn't remove the mapping. To do so, need to reindex! https://stackoverflow.com/a/38874129/12572612
## https://stackoverflow.com/a/16156873/12572612
# @ console @ https://32a112996b92429ca3c682984cdf5fc4.us-central1.gcp.cloud.es.io:9243/app/dev_tools#/console
# POST /nhat_test/_update_by_query?conflicts=proceed&pretty
# {
#  "script" : "ctx._source.remove('newField')"
# }
# https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.update_by_query
def delete_field_ES(sf):
    index_name = f"bossdata{sf}"
    ES_DEV.update_by_query(index = index_name, conflicts = "proceed", body = {
        "script" : "ctx._source.remove('tractTypeOZ')"
    })
    print(f"Removed tractTypeOZ fields from {index_name}")

    
# with Pool(60) as pool:
    # pool.map(download_ES, SF56.keys())
    
    # # KEEP TIMING OUT! ERROR: ConnectionTimeout caused by - ReadTimeoutError(None: None)   
    # pool.map(delete_field_ES, SF56.keys())
    
    ## RUN ONCE
    # pool.map(update_ES_mapping, SF56.keys())



# Add tractTypeKeywordOZ to df


In [479]:
# NOTE: use dtype=str to retain leading columns in tractNumber
oz_df = pd.read_excel('https://www.cdfifund.gov/sites/cdfi/files/documents/designated-qozs.12.14.18.xlsx', 
                      header=4, dtype=str)
oz_df.columns = ['state', 'county', 'tractID', 'tractTypeKeywordOZ', 'source']

oz_df.sort_values('tractID', inplace=True)
# oz_df['sf'] = oz_df.tractID.str[:2]
# 8764 unique OZ, 2 different types of OZ, OZ source is irrelevant
oz_df.nunique(), set(oz_df.tractTypeKeywordOZ), set(oz_df.source)

(state                   56
 county                1424
 tractID               8764
 tractTypeKeywordOZ       2
 source                   2
 dtype: int64,
 {'Low-Income Community', 'Non-LIC Contiguous'},
 {'2011-2015', '2012-2016'},
 (8764, 5),
      state   county      tractID    tractTypeKeywordOZ     source
 0  Alabama  Autauga  01001020700  Low-Income Community  2011-2015
 1  Alabama  Baldwin  01003010200  Low-Income Community  2011-2015
 2  Alabama  Baldwin  01003010400    Non-LIC Contiguous  2011-2015)

In [483]:
oz_df = oz_df[['tractID', 'tractTypeKeywordOZ']]
oz_df.dtypes, oz_df.shape, oz_df.sample(3)

(tractID               object
 tractTypeKeywordOZ    object
 dtype: object,
 (8764, 2),
           tractID    tractTypeKeywordOZ
 2020  13021010400  Low-Income Community
 3359  24033803508  Low-Income Community
 6656  72069180202  Low-Income Community)

In [545]:
def upload_ES_df(input_df, index_name, idCol = 'GEOID', es_instance = ES_DEV):
    # actiondf: details all actions, contains 5 cols: _id, doc, _index, _op_type, doc_as_upsert
    # use idCol as document _id
    actiondf = input_df[[idCol]].copy().rename(columns={idCol:'_id'})
    # a dict of all the data to be updated; does contain idCol which will be uploaded like a regular column
    actiondf['doc'] = input_df.to_dict('records')  
    del input_df
    # Some constants
    actiondf['_index'] = index_name
    actiondf['_op_type'] = 'update'
    actiondf['doc_as_upsert'] = True
    
    print(f"Start uploading {len(actiondf)} records to {index_name}")
    helpers.bulk(es_instance, actiondf.to_dict('records'))
    print(f"Completed uploading {len(actiondf)} records to {index_name}")

# Upload tractTypeKeywordOZ to Elasticsearch    
def upload_OZ(sf):
    index_name = f"bossdata{sf}"
    ## GEOID from bossdata* have missing values for bossdata66 and bossdata69
    ## df = pd.read_csv(f'cb-ids/{sf}.txt', dtype=str, header=None, names=['GEOID'])
    ## Use GEOID from tiger2019-tabblock10 instead 
    df = pd.read_csv(f'tiger-ids/{sf}.txt', dtype=str, header=None, names=['GEOID'])
    df['tractID'] = df.GEOID.str[:CT_LENGTH]
    
    df = df.merge(oz_df, how='left', on='tractID').drop(columns='tractID').fillna('isNotOZ')

    # ACTUAL UPLOAD
    if list(df.dtypes) == [object, object] and set(df) == {"GEOID", "tractTypeKeywordOZ"}: # Upload iff correct columns
        upload_ES_df(df, index_name)
    else:
        print(f"ALERT: bad df, no upload done: {sf=}, {df.shape=} \n")
        
    df['sf'] = sf
    return df    

with Pool(60) as pool:
    # res = pool.map(upload_OZ, SF56)   
    res = pool.map(upload_OZ, ["02", "22"])   
    oz_cbs = pd.concat(res, ignore_index=True)
# (11166336, 3)
# print(oz_cbs.shape, oz_cbs.dtypes)

oz_cbs.groupby(['sf', 'tractTypeKeywordOZ']).size().unstack(fill_value=0)

Start uploading 45292 records to bossdata02
Start uploading 204447 records to bossdata22
Completed uploading 45292 records to bossdata02
Completed uploading 204447 records to bossdata22


tractTypeKeywordOZ,Low-Income Community,Non-LIC Contiguous,isNotOZ
sf,,,
02,11539,0,33753
22,25686,1567,177194


In [382]:
# def num_blocks(sf):
#     tiger19_10 = gp.read_file(f'https://www2.census.gov/geo/tiger/TIGER2019/TABBLOCK/tl_2019_{sf}_tabblock10.zip')['GEOID10']
#     np.savetxt(f'tiger-ids/{sf}.txt', tiger19_10, fmt="%s")
#     tiger19_10 = len(tiger19_10)
#     print(f"Saved sf={sf}, len={tiger19_10} to file")
#     return 
#     # tiger20_10 = len(gp.read_file(f"https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK/tl_2020_{sf}_tabblock10.zip", ignore_geometry=True))
    
#     # #OFC, different from all others
#     # # tiger20_20 = len(gp.read_file(f'https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_{sf}_tabblock20.zip', ignore_geometry=True))
    
#     # es = len(pd.read_csv(f'cb-ids/{sf}.txt', dtype=str, header=None, names=['GEOID']))
    
#     # return sf, tiger19_10, tiger20_10, es

# with Pool(30) as pool:
#     res = pool.map(num_blocks, SF56)


# TIGER2019-CB vs 2019-CBG vs 2019-tracts, and <> ES

In [419]:
## DOWNLOAD
def num_cbg(sf):
    df = gp.read_file(f'https://www2.census.gov/geo/tiger/TIGER2019/BG/tl_2019_{sf}_bg.zip', ignore_geometry=True)['GEOID']
    np.savetxt(f'tiger-cbg10/{sf}.txt', df, fmt="%s")

with Pool(60) as pool:
    pool.map(num_cbg, SF56)

def num_tracts(sf):
    df = gp.read_file(f'https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_{sf}_tract.zip', ignore_geometry=True)['GEOID']
    np.savetxt(f'tiger-tracts10/{sf}.txt', df, fmt="%s")

with Pool(60) as pool:
    pool.map(num_tracts, SF56)    

# READ and find diff

In [4]:
def diff_GEOID(sf):
    es = pd.read_csv(f'cb-ids/{sf}.txt', dtype=str, header=None, names=['ES'])
    tiger = pd.read_csv(f'tiger-ids/{sf}.txt', dtype=str, header=None, names=['TIGER'])
    df = es.merge(tiger, how='outer', left_on="ES", right_on="TIGER")
    df['sf'] = sf
    return df

with Pool(60) as pool:
    data = pool.map(diff_GEOID, SF56)
    all_cbs = pd.concat(data, ignore_index=True)

all_cbs['ESCBG'] = all_cbs.ES.str[:CBG_LENGTH]
all_cbs['ESCT'] = all_cbs.ES.str[:CT_LENGTH]    
all_cbs['TIGERCBG'] = all_cbs.TIGER.str[:CBG_LENGTH]
all_cbs['TIGERCT'] = all_cbs.TIGER.str[:CT_LENGTH]   
all_cbs.shape

(11166336, 7)

In [563]:
## IF NOT, should we redo merge-upload above, using join='outer' instead? 
## No, because those OZ tracts don't have their contained CB info
## i.e. cannot represent them on bossdata* with any GEOID!

## @yuan: NOTE! OZ {'51019050100'} not found in TIGER GEOID (TIGER2019-tabblock10-truncated)
set(oz_df.tractID) - set(all_cbs.TIGERCT)

{'51019050100'}

2022-04-26 12:16:15,965 - distributed.diskutils - ERROR - Failed to remove '/home/nhat/opportunity-zones/dask-worker-space/worker-665zbcxg' (failed in <built-in function lstat>): [Errno 2] No such file or directory: '/home/nhat/opportunity-zones/dask-worker-space/worker-665zbcxg'
2022-04-26 12:16:17,138 - distributed.diskutils - ERROR - Failed to remove '/home/nhat/opportunity-zones/dask-worker-space/worker-keshbfd9' (failed in <built-in function lstat>): [Errno 2] No such file or directory: '/home/nhat/opportunity-zones/dask-worker-space/worker-keshbfd9'
2022-04-26 12:16:17,138 - distributed.diskutils - ERROR - Failed to remove '/home/nhat/opportunity-zones/dask-worker-space/worker-2wbx9r12' (failed in <built-in function lstat>): [Errno 2] No such file or directory: '/home/nhat/opportunity-zones/dask-worker-space/worker-2wbx9r12'
2022-04-26 12:16:17,139 - distributed.diskutils - ERROR - Failed to remove '/home/nhat/opportunity-zones/dask-worker-space/worker-ryql4gw7' (failed in <built

In [10]:
# At sf= 66, and 69: bossdata* have missing TIGER-GEOIDs
## I have added these to bossdata* as of Apr 25.
all_cbs.query("ES != ES").sf.unique()

array(['66', '69'], dtype=object)

time: 3.26 s (started: 2022-04-28 23:27:35 -04:00)


In [13]:
## READ ids from TIGER2019-cbg and TIGER2019-ct
def diff_GEOID(sf):
    cbg = pd.read_csv(f'tiger-cbg/{sf}.txt', dtype=str, header=None, names=['CBG'])
    ct = pd.read_csv(f'tiger-ct/{sf}.txt', dtype=str, header=None, names=['CT'])
    return cbg, ct

with Pool() as pool:
    res = pool.map(diff_GEOID, SF56)
    cbgs = pd.concat([cbg for cbg, _ in res], ignore_index=True)
    cts = pd.concat([ct for _, ct in res], ignore_index=True)


time: 1.19 s (started: 2022-04-28 23:29:00 -04:00)


In [14]:
## Merge with TIGER2019-geoid
all_cbs = all_cbs.merge(cbgs, how='outer', left_on="TIGERCBG", right_on="CBG")
all_cbs = all_cbs.merge(cts, how='outer', left_on="TIGERCT", right_on="CT")


time: 7.24 s (started: 2022-04-28 23:29:04 -04:00)


In [ ]:
# TIGER-CB truncated to CBG and CT
# diff <> TIGER-CBG, <> TIGER-CT!

In [15]:
all_cbs1 = all_cbs.copy()

ES          4095
TIGER         97
TIGERCBG      97
TIGERCT       97
CBG         6321
CT          6337
dtype: int64

time: 9.87 s (started: 2022-04-28 23:29:12 -04:00)


In [58]:
diff = all_cbs[all_cbs.isna().any(1)]
all_cbs.isnull().sum()

ES             4098
TIGER           100
sf              100
ESCBG          4098
ESCT           4098
TIGERCBG        100
TIGERCT         100
CBG            6324
CT             6340
TIGERCOUNTY    4098
GEOID          6872
dtype: int64

time: 12.8 s (started: 2022-04-28 23:55:58 -04:00)


# COUNTY diff

In [50]:
counties = gp.read_file("https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip", ignore_geometry=True)['GEOID']

time: 1.92 s (started: 2022-04-28 23:49:55 -04:00)


In [52]:
all_cbs['TIGERCOUNTY'] = all_cbs.ES.str[:COUNTY_LENGTH]
## Merge with TIGER2019-COUNTY
all_cbs = all_cbs.merge(counties, how='outer', left_on="TIGERCOUNTY", right_on="GEOID")

time: 3.41 s (started: 2022-04-28 23:53:09 -04:00)


In [56]:
all_cbs

,ES,TIGER,sf,ESCBG,ESCT,TIGERCBG,TIGERCT,CBG,CT,TIGERCOUNTY,GEOID
0,011270215002009,011270215002009,01,011270215002,01127021500,011270215002,01127021500,011270215002,01127021500,01127,01127
1,011270215002012,011270215002012,01,011270215002,01127021500,011270215002,01127021500,011270215002,01127021500,01127,01127
2,011270215002030,011270215002030,01,011270215002,01127021500,011270215002,01127021500,011270215002,01127021500,01127,01127
3,011270215002031,011270215002031,01,011270215002,01127021500,011270215002,01127021500,011270215002,01127021500,01127,01127
4,011270215002036,011270215002036,01,011270215002,01127021500,011270215002,01127021500,011270215002,01127021500,01127,01127
...,...,...,...,...,...,...,...,...,...,...,...
11166431,780309900000009,780309900000009,78,780309900000,78030990000,780309900000,78030990000,780309900000,78030990000,78030,78030
11166432,780309900000013,780309900000013,78,780309900000,78030990000,780309900000,78030990000,780309900000,78030990000,78030,78030
11166433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69085
11166434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02158


time: 10.2 ms (started: 2022-04-28 23:54:20 -04:00)


In [78]:
diff1 = diff.query("TIGER.notna() & CBG.isna() & GEOID.notna() & CT.notna()", engine="python")
diff1

,ES,TIGER,sf,ESCBG,ESCT,TIGERCBG,TIGERCT,CBG,CT,TIGERCOUNTY,GEOID
1063143,060378002043013,060378002043013,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063144,060378002043023,060378002043023,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063145,060378002043011,060378002043011,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063146,060378002043027,060378002043027,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063147,060378002043007,060378002043007,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063148,060378002043029,060378002043029,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063149,060378002043025,060378002043025,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063150,060378002043005,060378002043005,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063151,060378002043017,060378002043017,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037
1063152,060378002043024,060378002043024,06,060378002043,06037800204,060378002043,06037800204,NaN,06037800204,06037,06037


time: 16.2 ms (started: 2022-04-29 16:18:36 -04:00)


In [60]:
diff1 = diff.query("TIGERCOUNTY.isna() & ~GEOID.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.GEOID)

((3, 11), {'02158', '46102', '69085'})

time: 6.54 ms (started: 2022-04-28 23:56:40 -04:00)


In [74]:
"36085" in set(all_cbs.TIGERCOUNTY), "36085" in set(counties)

(True, True)

time: 711 ms (started: 2022-04-29 00:02:14 -04:00)


## in GEOID, but not in CBG or CT

In [36]:
diff1 = diff.query("~TIGER.isna() & CBG.isna()", engine="python")
 # (6296, 6)
print(diff1.shape, set(diff1.TIGER.str[:5]))

(6296, 6) {'69120', '04019', '06037', '02270', '36065', '36053', '46113', '51515', '36085'}
time: 8.75 ms (started: 2022-04-28 23:33:45 -04:00)


In [39]:
diff1 = diff.query("~TIGER.isna() & CT.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.TIGER.str[:5])

((6265, 6),
 {'02270', '04019', '06037', '36053', '36065', '36085', '46113', '51515'})

time: 10.1 ms (started: 2022-04-28 23:35:25 -04:00)


## not in GEOID, but in CBG or CT

In [32]:
diff1 = diff.query("TIGER.isna() & ~CBG.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.CBG.str[:5])

((72, 6), {'02158', '04019', '06037', '36053', '36065', '46102', '51019'})

time: 6.75 ms (started: 2022-04-28 23:32:55 -04:00)


In [42]:
diff1 = diff.query("TIGER.isna() & ~CT.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.CT.str[:5])

((25, 6), {'02158', '04019', '06037', '36053', '36065', '46102', '51019'})

time: 6.75 ms (started: 2022-04-28 23:36:41 -04:00)


"Wade Hampton Census Area, Alaska" (geo code AK270, FIPS code 02270) was renamed and recoded as "Kusilvak Census Area" (AK158, 02158) effective July 1, 2015.
"Shannon County, South Dakota" (SD113, 46113) was renamed and recoded as "Oglala Lakota County" (SD102, 46102) effective May 1, 2015.
The independent city of "Bedford, Virginia" (VA515, 51515) was merged into "Bedford County" (VA019, 51019) effective July 1, 2013, with an estimated net added population of 6,222.  This change deprecates the BEA combined area of "Bedford and Bedford City" (VA909).
https://www.economy.com/support/blog/buffet.aspx?did=EBD51E7D-8822-42F2-BD0D-8937239911E9


In [ ]:
FROM (found in TIGER-GEOID)
['02270', '36085', '46113', '51515', ]

TO (found in TIGER-CBG, and TIGER-CT)
['02158',          '46102', '51019', ]

In [45]:
geoid_cbg = {'69120', '04019', '06037', '02270', '36065', '36053', '46113', '51515', '36085'}
geoid_ct =  {'02270', '04019', '06037', '36053', '36065', '36085', '46113', '51515'}
geoid_cbg - geoid_ct, geoid_ct - geoid_cbg # ({'69120'}, set())

cbg_geoid = {'02158', '04019', '06037', '36053', '36065', '46102', '51019'}
ct_geoid = {'02158', '04019', '06037', '36053', '36065', '46102', '51019'}
cbg_geoid - ct_geoid, ct_geoid - cbg_geoid  # (set(), set())


cbg_ct = {'02158', '04019', '06037', '36053', '36065', '46102', '51019'}
ct_cbg = {'02158', '04019', '06037', '36053', '36065', '46102', '51019', '69120'}
cbg_ct - ct_cbg, ct_cbg - cbg_ct # (set(), {'69120'})

(set(), {'69120'})

time: 3.02 ms (started: 2022-04-28 23:39:40 -04:00)


## in/not in CBG vs CT

In [34]:
diff1 = diff.query("CT.isna() & ~CBG.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.CBG.str[:5])

((72, 6), {'02158', '04019', '06037', '36053', '36065', '46102', '51019'})

time: 6.22 ms (started: 2022-04-28 23:33:21 -04:00)


In [35]:
diff1 = diff.query("CBG.isna() & ~CT.isna()", engine="python")
 # (6295, 6)
diff1.shape, set(diff1.CT.str[:5])

((56, 6),
 {'02158', '04019', '06037', '36053', '36065', '46102', '51019', '69120'})

time: 6.23 ms (started: 2022-04-28 23:33:32 -04:00)


# EXTRAS

In [42]:
# all current bossdata indices on ES
indices = """bossdata01 
bossdata02 
bossdata04 
bossdata05 
bossdata06 
bossdata08 
bossdata09 
bossdata10 
bossdata11 
bossdata12 
bossdata13 
bossdata15 
bossdata16 
bossdata17 
bossdata18 
bossdata19 
bossdata20 
bossdata21 
bossdata22 
bossdata23 
bossdata24 
bossdata25 
bossdata26 
bossdata27 
bossdata28 
bossdata29 
bossdata30 
bossdata31 
bossdata32 
bossdata33 
bossdata34 
bossdata35 
bossdata36 
bossdata37 
bossdata38 
bossdata39 
bossdata40 
bossdata41 
bossdata42 
bossdata44 
bossdata45 
bossdata46 
bossdata47 
bossdata48 
bossdata49 
bossdata50 
bossdata51 
bossdata53 
bossdata54 
bossdata55 
bossdata56 
bossdata60 
bossdata66 
bossdata69 
bossdata72 
bossdata78""".split()
indices = {x[-2:] for x in indices}
set(SF56) - indices, indices - set(SF56)

(set(), set())

In [44]:
# https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
#  List of all National FIPS and GNIS Codes File SF57
# sf_df = pd.read_csv('https://www2.census.gov/geo/docs/reference/state.txt', 
#                     dtype=str, sep='|')
# sf_df.columns = ['statefips', 'code', 'name', 'ENS']
# # THERE ARE 57 STATEFIPS IN TOTAL! PLUS MORE
# FIPS States Codes for the Outlying Areas of the United States and the Freely Associated States
# SF57 = sf_df.set_index('statefips').to_dict()['code']
# sf_df.shape, len(SF57)
# print(SF57)

### all OZ are found in SF56
oz_codes = set(oz_df.sf)
# sf_df.query('statefips == "74"')
# 74	UM	U.S. Minor Outlying Islands
set(SF56) - oz_codes, oz_codes - set(SF56)



(set(), set())